In [1]:
import os.path
import bs4 as bs
import urllib.request
import glob
import pandas as pd

In [2]:
cols_all = ["Match_Index", "Team_Home", "Team_Away", "Minute", "Goals_Home", "Goals_Away", "Subs_Home", "Subs_Away",
        "Event", "Event_Team","Current_Points","Final_Points", "Event_Coach"]

cols_subs = ["Match_Index", "Team_Home", "Team_Away", "Minute", "Sub_Number", "Event_Team", "Goals_Home", "Goals_Away","Current_Points","Final_Points", "Event_Coach"]

In [8]:
season = "2021-22"  # season to analyze
default_url = f"https://www.kicker.de/bundesliga/spieltag/{season}/"

start_match_day = 1
end_match_day = 27
for i in range(start_match_day,end_match_day+1):  # for every match day in given range...
    # ...get link to every match
    url =  default_url+str(i)

    kicker_Client = urllib.request.urlopen(url)
    kicker_html = kicker_Client.read()
    kicker_Client.close()

    kicker_Soup = bs.BeautifulSoup(kicker_html, "lxml")

    match_container = kicker_Soup.findAll("a",{"class":"kick__v100-scoreBoard kick__v100-scoreBoard--standard"})

    for k in range(len(match_container)):   # for every match of math day...
        link = match_container[k]["href"]
        # ...go to "Schema"-page of kicker article
        link = "https://www.kicker.de"+str(link).replace("analyse","schema")

        if "schema" in link:
            match_index = link.split("-").pop()[:7]
            if match_index == "4721736":    # match index of abandoned match Bochum - M'Gladbach
                continue

            # scrape page
            match_Client = urllib.request.urlopen(link)
            match_html = match_Client.read()
            match_Client.close()
            match_Soup = bs.BeautifulSoup(match_html, "lxml")

            # get final score
            final_goals = match_Soup.findAll("div", {"kick__v100-scoreBoard__scoreHolder__score"})
            final_home_goals = int(final_goals[0].text)
            final_away_goals = int(final_goals[1].text)

            # get home and away coaches and initialize goal and sub counters
            coach_container = match_Soup.find("section",{"class":"kick__section-item kick__section-item--sec"})
            coaches = coach_container.findAll("div",{"class":"kick__lineup-text"})

            home = coaches[0].text.split("\n")
            home = list(filter(None, home))
            home_team = home[0]
            home_coach = home[1]
            home_goals = 0
            home_subs = 0
            away = coaches[1].text.split("\n")
            away = list(filter(None, away))
            away_team = away[0]
            away_coach = away[1]
            away_goals = 0
            away_subs = 0
            current_points = 1

            final_home_points = final_away_points = 1
            if final_home_goals > final_away_goals:
                final_home_points = 3
                final_away_points = 0
            if final_away_goals > final_home_goals:
                final_home_points = 0
                final_away_points = 3

            # get all events of timeline
            event_container = match_Soup.find("div", {"class":"kick__game-timeline"})
            events = event_container.findAll("div",{"class":"kick__ticker-event-overlay-box kick__js_overlay-card kick__ticker-event-overlay-box--with-player"})

            match_events = []   # list of all events in match
            subs_events = []    # list of all substitutions in match
            for event in events:    # for all timestamps in timeline...
                text = event.text
                # ...get all events at timestamp (multiple events at single timestamp possible)
                goals = text.count("Tor") - text.count("Tor annulliert")
                subs = text.count("Spielerwechsel")
                reds = text.count("Rote Karte")
                yellows = text.count("Gelbe Karte")
                minute = text[8:].split(".")[0]
                if home_goals == away_goals:
                    current_points = 1
                if home_team in text:   # home team event
                    if home_goals > away_goals:
                        current_points = 3
                    if home_goals < away_goals:
                        current_points = 0
                    for goal in range(goals):
                        home_goals += 1
                        match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Tor",home_team, current_points,final_home_points,home_coach))
                    for sub in range(subs):
                        home_subs += 1
                        match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Wechsel",home_team,current_points,final_home_points,home_coach))
                        subs_events.append((match_index, home_team, away_team, minute, home_subs, home_team, home_goals,
                                    away_goals, current_points,final_home_points, home_coach))
                    for red in range(reds):
                        match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Rote Karte",home_team,current_points,final_home_points,home_coach))
                    for yellow in range(yellows):
                        match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Gelbe Karte",home_team,current_points,final_home_points,home_coach))
                else:   # away team event
                    if home_goals < away_goals:
                        current_points = 3
                    if home_goals > away_goals:
                        current_points = 0
                    for goal in range(goals):
                        away_goals += 1
                        match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Tor",away_team,current_points,final_away_points,away_coach))
                    for sub in range(subs):
                        away_subs += 1
                        match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Wechsel",away_team,current_points,final_away_points,away_coach))
                        subs_events.append((match_index, home_team, away_team, minute, away_subs, away_team, home_goals,
                                    away_goals, current_points,final_away_points, away_coach))
                    for red in range(reds):
                        match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Rote Karte",away_team,current_points,final_away_points,away_coach))
                    for yellow in range(yellows):
                        match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Gelbe Karte",away_team,current_points,final_away_points,away_coach))

            # create directories if not existent
            if not os.path.exists('all_events'):
                os.makedirs('all_events')
            if not os.path.exists('subs'):
                os.makedirs('subs')

            # save data as csv-file
            pd.DataFrame(match_events,columns=cols_all).to_csv(f"all_events/{match_index}.csv", index=False, encoding='utf-8')
            pd.DataFrame(subs_events,columns=cols_subs).to_csv(f"subs/{match_index}.csv", index=False, encoding='utf-8')

In [15]:
extension = 'csv'

current_path = os.getcwd()

# combine csv-files to single csv-file
os.chdir("all_events")
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]    # get names of all csv-files in directory
all_events = pd.concat([pd.read_csv(f) for f in all_filenames ])    # concatenate dataframes to single one
all_events.to_csv("all_events.csv", index=False, encoding='utf-8')  # saves combined data as csv-file

os.chdir(current_path)

os.chdir("subs")
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_subs = pd.concat([pd.read_csv(f) for f in all_filenames ])
all_subs.to_csv("all_subs.csv", index=False, encoding='utf-8')

os.chdir(current_path)